In [7]:
def stock_pipeline(user_input):
    # notebooks/Project_3a.ipynb

    # Import system references
    import sys
    import os

    # Ensure project_root is in the system path
    current_dir = os.getcwd()
    project_root = os.path.abspath(os.path.join(current_dir, '..'))
    if project_root not in sys.path:
        sys.path.append(project_root)
        
    #print("Project root added to sys.path:", project_root in sys.path)
    #print(sys.path)

    #setup Logging
    import logging

    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.StreamHandler()
        ]
    )

    logger = logging.getLogger(__name__)

    # Configure logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    #setup Logging
    import logging

    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.StreamHandler()
        ]
    )

    logger = logging.getLogger(__name__)

    # Configure logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


    # Import necessary libraries
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns

    from datetime import datetime
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    import torch
    import torch.nn as nn
    import torch.optim as optim

    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense
    from tensorflow.keras.optimizers import Adam
    from sklearn.preprocessing import MinMaxScaler

    # Import the fetch_stock_data function
    from scripts import fetch_stock_data, transform_stock_data_to_delta, transform_with_history
    from scripts import prepare_data_for_training, create_time_series_windows, lstm_model, prepare_sliding_window_data

    # Ask the user for a ticker symbol
    ticker = user_input

    # Define start and end dates
    start_date = '2021-01-01'

    # If end_date is not provided, you can set it to today's date
    end_date = datetime.today().strftime('%Y-%m-%d')  # Default to today's date if not specified

    # Print the ticker and date range to confirm
    print(f"Fetching data for {ticker} from {start_date} to {end_date}")

    # Fetch stock data
    stock_data_df = fetch_stock_data(ticker, start_date, end_date)
    #stock_data_df = fetch_stock_data(ticker, start_date)

    # Check if data is fetched successfully
    if not stock_data_df.empty:
        # Display the first few rows
        display(stock_data_df.head())
    else:
        print("No data to display.")


    # Transform Stock Data to Deltas
    columns_to_exclude = ['Adj Close']  # Drop 'Adj Close'
    columns_to_keep = []  # Keep 'Volume' but exclude from delta calculation
    columns_to_calculate = ['Open', 'High', 'Low', 'Close', 'Volume']  # Calculate deltas for 'Open' and 'Close'

    transformed_data_df = transform_stock_data_to_delta(
        stock_data_df, 
        columns_to_exclude=columns_to_exclude, 
        columns_to_calculate=columns_to_calculate, 
        columns_to_keep=columns_to_keep
    )

    #transformed_data_df = transform_stock_data_to_delta(stock_data_df)
    #transformed_data_df = transform_stock_data_to_delta(stock_data_df, exclude=['Volume'])

    # Display Transformed Data
    if not transformed_data_df.empty:
        display(transformed_data_df.head())
    else:
        print("No transformed data to display.")

    # Prepare the data by removing non-delta columns
    delta_only_df = prepare_sliding_window_data(transformed_data_df)

    # Display the first few rows of the resulting DataFrame
    display(delta_only_df.head())

    # Define parameters
    n_timesteps = 100  # Number of timesteps (sequence length)

    # Step 1: Create the time series windows (X and y)
    X, y = create_time_series_windows(delta_only_df, 'Close_delta', n_timesteps)

    # Display the first few occurrences of the X and y arrays
    print("First 5 entries of X:\n", X[:5])  # Display first 5 rows
    print("First 5 entries of y:\n", y[:5])  # Display first 5 target values

    # Step 2: Split the data into training and testing sets (80/20 split)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

    # Display the shapes of the training and testing sets
    print("X_train shape:", X_train.shape)
    print("X_test shape:", X_test.shape)
    print("y_train shape:", y_train.shape)
    print("y_test shape:", y_test.shape)

    # Initialize the scaler for X and y values
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    # Reshape X data to 2D for scaling, keeping the last dimension as features
    n_samples_train = X_train.shape[0]
    n_samples_test = X_test.shape[0]

    X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])  # Reshape to 2D: [samples * timesteps, features]
    X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

    # Apply scaling to X features (fit on X_train, transform both X_train and X_test)
    X_train_scaled = scaler_X.fit_transform(X_train_reshaped).reshape(n_samples_train, X_train.shape[1], X_train.shape[2])
    X_test_scaled = scaler_X.transform(X_test_reshaped).reshape(n_samples_test, X_test.shape[1], X_test.shape[2])

    # Reshape y values to 2D (required by MinMaxScaler)
    y_train = y_train.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)

    # Apply scaling to y values
    y_train_scaled = scaler_y.fit_transform(y_train)
    y_test_scaled = scaler_y.transform(y_test)


    # Define parameters
    #n_timesteps = 25
    n_features = X_train_scaled.shape[2]

    # Define the number of layers and units per layer
    num_layers = 3
    units_per_layer = [50, 100, 50]  # 3 layers with 50, 100, and 50 units, respectively

    # Call the LSTM model
    y_pred_scaled, model = lstm_model(X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, n_timesteps, n_features, num_layers, units_per_layer, 
                            learning_rate=0.00001, epochs=70, batch_size=28)
    # Output the predictions
    #print("Predictions from LSTM model:", y_pred_scaled)

    # Step 1: Inverse transform the predicted and actual y values to the original scale
    y_pred_original = scaler_y.inverse_transform(y_pred_scaled)
    y_test_original = scaler_y.inverse_transform(y_test_scaled)

    # Display the first few occurrences of prediction and test scaled and unscaled arrays
    print("First 5 entries of y_pred_scaled", y_pred_scaled[:5])  # Display first 5 rows
    print("First 5 entries of y_pred_original", y_pred_original[:5])  # Display first 5 rows
    print("First 5 entries of y_test_scaled", y_test_scaled[:5])  # Display first 5 target values
    print("First 5 entries of y_test_original", y_test_original[:5])  # Display first 5 target values

    # Mean Squared Error
    mse = mean_squared_error(y_test_original, y_pred_original)
    # Root Mean Squared Error
    rmse = np.sqrt(mse)
    # Mean Absolute Error
    mae = mean_absolute_error(y_test_original, y_pred_original)
    # R-squared
    r2 = r2_score(y_test_original, y_pred_original)

    # Print the metrics
    print(f"Mean Squared Error (MSE): {mse:.6f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.6f}")
    print(f"Mean Absolute Error (MAE): {mae:.6f}")
    print(f"R-squared (R²): {r2:.6f}")


    # Create a DataFrame to compare actual and predicted values
    comparison_df = pd.DataFrame({'Actual': y_test_original.flatten(), 'Predicted': y_pred_original.flatten()})
    # Calculate the difference (error)
    comparison_df['Difference'] = comparison_df['Actual'] - comparison_df['Predicted']
    #print(comparison_df.head(25))


    # Get the sign of the actual and predicted values
    actual_sign = np.sign(comparison_df['Actual'])
    predicted_sign = np.sign(comparison_df['Predicted'])
    #print(actual_sign.head(25))
    #print(predicted_sign.head(25))
    # Check where the signs match
    sign_matches = actual_sign == predicted_sign
    print(sign_matches.head(100))

    # Calculate the percentage of sign matches
    percentage_match = sign_matches.mean() * 100

    print(f"Percentage of Sign Matches: {percentage_match :.2f}%")


    # Get the prediction for tomorrow's Close_delta

    """
    Predict the next Close_delta based on the most recent n_timesteps of data.
        
    Using:
    - model: The trained LSTM model.
    - delta_only_df (pd.DataFrame): The DataFrame containing all historical data, including the latest Close_delta.
    - n_timesteps (int): The number of timesteps (sequence length) used in the model.
    - scaler_X: The scaler used to scale the features (X).
    - scaler_y: The scaler used to scale the target (y).

    Returns:
    - float: The predicted Close_delta for the next day.
    """
    # Step 1: Extract the last n_timesteps rows from the data (to be used as input for prediction)
    last_window = delta_only_df[-n_timesteps:].values.reshape(1, n_timesteps, delta_only_df.shape[1])

    # Step 2: Scale the input features
    last_window_scaled = scaler_X.transform(last_window.reshape(-1, last_window.shape[-1])).reshape(1, n_timesteps, -1)

    # Step 3: Use the model to predict the next Close_delta (scaled)
    predicted_close_delta_scaled = model.predict(last_window_scaled)

    # Step 4: Inverse transform the prediction to get the original scale of Close_delta
    predicted_close_delta = scaler_y.inverse_transform(predicted_close_delta_scaled)

    predicted_close_delta = predicted_close_delta[0][0]  # Return the predicted value

    # Decision logic: Buy if positive, Sell if negative
    if predicted_close_delta > 0:
        print(f"Predicted Close increase for {ticker} tomorrow is {(predicted_close_delta * 100):.2f}%. Suggestion: BUY with {percentage_match :.2f}% confidence")
    else:
        print(f"Predicted Close decrease for {ticker} tomorrow is {(predicted_close_delta * 100):.2f}%. Suggestion: SELL with {percentage_match :.2f}% confidence")

In [9]:
stock_pipeline('AAPL')

2024-10-28 20:40:47,406 - INFO - NumExpr defaulting to 8 threads.
2024-10-28 20:40:57,664 - INFO - End date provided: 2024-10-28
2024-10-28 20:40:57,665 - INFO - Fetching data for ticker: AAPL from 2021-01-01 to 2024-10-28


Fetching data for AAPL from 2021-01-01 to 2024-10-28


2024-10-28 20:40:59,053 - INFO - Resetting index to make 'Date' a column.
2024-10-28 20:40:59,055 - INFO - Flattening multi-level column names (removing ticker symbol).
2024-10-28 20:40:59,055 - INFO - Successfully fetched and simplified data for ticker 'AAPL'.


Price,Date,Adj Close,Close,High,Low,Open,Volume
0,2021-01-04 00:00:00+00:00,126.683449,129.410004,133.610001,126.760002,133.520004,143301900
1,2021-01-05 00:00:00+00:00,128.249725,131.009995,131.740005,128.429993,128.889999,97664900
2,2021-01-06 00:00:00+00:00,123.932663,126.599998,131.050003,126.379997,127.720001,155088000
3,2021-01-07 00:00:00+00:00,128.161606,130.919998,131.630005,127.860001,128.360001,109578200
4,2021-01-08 00:00:00+00:00,129.267838,132.050003,132.630005,130.229996,132.429993,105158200


2024-10-28 20:40:59,070 - INFO - Starting transformation of stock data to deltas.
2024-10-28 20:40:59,071 - INFO - Dropping columns: ['Adj Close']
2024-10-28 20:40:59,074 - INFO - Calculating deltas for specified columns: ['Open', 'High', 'Low', 'Close', 'Volume']
2024-10-28 20:40:59,075 - INFO - Transforming column: Open
2024-10-28 20:40:59,078 - INFO - Transforming column: High
2024-10-28 20:40:59,078 - INFO - Transforming column: Low
2024-10-28 20:40:59,088 - INFO - Transforming column: Close
2024-10-28 20:40:59,093 - INFO - Transforming column: Volume
2024-10-28 20:40:59,095 - INFO - Dropping the first row with NaN values after delta calculation.
2024-10-28 20:40:59,100 - INFO - Successfully transformed stock data to deltas.


Price,Date,Close,High,Low,Open,Volume,Open_delta,High_delta,Low_delta,Close_delta,Volume_delta
1,2021-01-05 00:00:00+00:00,131.009995,131.740005,128.429993,128.889999,97664900,-0.0347,-0.0140,0.0132,0.0124,-0.3185
2,2021-01-06 00:00:00+00:00,126.599998,131.050003,126.379997,127.720001,155088000,-0.0091,-0.0052,-0.0160,-0.0337,0.5880
3,2021-01-07 00:00:00+00:00,130.919998,131.630005,127.860001,128.360001,109578200,0.0050,0.0044,0.0117,0.0341,-0.2934
4,2021-01-08 00:00:00+00:00,132.050003,132.630005,130.229996,132.429993,105158200,0.0317,0.0076,0.0185,0.0086,-0.0403
5,2021-01-11 00:00:00+00:00,128.979996,130.169998,128.500000,129.190002,100384500,-0.0245,-0.0185,-0.0133,-0.0232,-0.0454


Price,Open_delta,High_delta,Low_delta,Close_delta,Volume_delta
1,-0.0347,-0.0140,0.0132,0.0124,-0.3185
2,-0.0091,-0.0052,-0.0160,-0.0337,0.5880
3,0.0050,0.0044,0.0117,0.0341,-0.2934
4,0.0317,0.0076,0.0185,0.0086,-0.0403
5,-0.0245,-0.0185,-0.0133,-0.0232,-0.0454


c:\Users\ryanw\anaconda3\envs\RyanDev2\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


First 5 entries of X:
 [[[-3.470e-02 -1.400e-02  1.320e-02  1.240e-02 -3.185e-01]
  [-9.100e-03 -5.200e-03 -1.600e-02 -3.370e-02  5.880e-01]
  [ 5.000e-03  4.400e-03  1.170e-02  3.410e-02 -2.934e-01]
  ...
  [ 1.440e-02  3.000e-03  3.000e-03 -1.600e-03  1.413e-01]
  [-6.700e-03 -7.200e-03  8.000e-04 -4.000e-04 -2.143e-01]
  [-4.100e-03  2.000e-03 -1.060e-02 -1.240e-02  6.725e-01]]

 [[-9.100e-03 -5.200e-03 -1.600e-02 -3.370e-02  5.880e-01]
  [ 5.000e-03  4.400e-03  1.170e-02  3.410e-02 -2.934e-01]
  [ 3.170e-02  7.600e-03  1.850e-02  8.600e-03 -4.030e-02]
  ...
  [-6.700e-03 -7.200e-03  8.000e-04 -4.000e-04 -2.143e-01]
  [-4.100e-03  2.000e-03 -1.060e-02 -1.240e-02  6.725e-01]
  [-6.900e-03 -1.440e-02 -4.200e-03 -5.300e-03 -2.464e-01]]

 [[ 5.000e-03  4.400e-03  1.170e-02  3.410e-02 -2.934e-01]
  [ 3.170e-02  7.600e-03  1.850e-02  8.600e-03 -4.030e-02]
  [-2.450e-02 -1.850e-02 -1.330e-02 -2.320e-02 -4.540e-02]
  ...
  [-4.100e-03  2.000e-03 -1.060e-02 -1.240e-02  6.725e-01]
  [-6.900e-

**Sentiment Analysis Pipeline**

In [20]:
from transformers import pipeline
import yfinance as yf 
import numpy as np 
import tensorflow as tf



In [22]:
# Initialize the sentiment analysis pipeline from Hugging Face
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_analysis = pipeline("sentiment-analysis",model=model_name)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

c:\Users\ryanw\anaconda3\envs\RyanDev2\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ryanw\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


RuntimeError: Failed to import transformers.models.distilbert.modeling_tf_distilbert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
# Function to fetch recent stock-related news using yfinance
def fetch_stock_news(stock_symbol, num_articles=5):
    stock = yf.Ticker(stock_symbol)
    news = stock.news  # Get stock news

    if not news:
        return []  # Return an empty list if no news is available

    #Change num_articles to int for Gradio use

    num_articles = int(num_articles)
    # Extract top `num_articles` news articles
    news_headlines = [article['title'] for article in news[:num_articles]]
    return news_headlines

In [ ]:
# Perform sentiment analysis on stock news headlines
def analyze_sentiment_on_news(stock_symbol, num_articles):
    # Fetch stock news
    headlines = fetch_stock_news(stock_symbol, num_articles)

    # Check if headlines are available
    if not headlines:
        print(f"No news found for {stock_symbol}")
        return

    #print(f"News headlines for {stock_symbol}:\n")

    # Perform sentiment analysis on each headline
    results = sentiment_analysis(headlines)

    # Initialize lists to store sentiment and scores
    sentiments = []
    scores = []

    for result in results:
        score = result['score']
        scores.append(score)
        sentiment = result['label']
        if sentiment == 'POSITIVE':
            sentiments.append(1)
        elif sentiment == 'NEGATIVE':
            sentiments.append(-1)
        else:
            sentiments.append(0)



    # Calculate the average sentiment score
    average_sentiment = np.mean(sentiments)

    # Determine overall sentiment
    if average_sentiment > 0:
      overall_sentiment = "Positive"
    elif average_sentiment < 0:
      overall_sentiment = "Negative"
    else:
      overall_sentiment = "Neutral"

        # Print results
    def average(numbers):
     sum_of_numbers = sum(numbers)
     average = (sum_of_numbers / len(numbers)) * 100
     return average


    number = average(scores)

    #formatted_number = number.replace(",", "")
    #formatted_overall_sentiment = overall_sentiment.replace(",", "")
    #return formatted_number, formatted_overall_sentiment

    return f"{number:.2f}% | {overall_sentiment}"


    # Print results
    #for i, headline in enumerate(headlines):
        #sentiment = results[i]['label']
        #score = results[i]['score']
        #print(f"Headline: {headline}")
        #print(f"Sentiment: {sentiment}, Score: {score:.4f}")
        #print("----------------------------------------------------")

In [ ]:
 # Example: Analyze sentiment on Apple (AAPL) stock news
stock_symbol = "AAPL"
num_articles = 5
analyze_sentiment_on_news(stock_symbol, num_articles)

**Combined Pipelines**

In [ ]:
def combined_pipelines(ticker, num_articles=None):
  prediction_one = stock_pipeline(ticker)
  if num_articles:
    prediction_two = analyze_sentiment_on_news(ticker,num_articles)
  else:
    prediction_two = analyze_sentiment_on_news(ticker,5)
  return prediction_one, prediction_two

**Gradio Interface**

In [ ]:
import gradio as gr

input_component_one = gr.Textbox(label="Input sequence (comma-separated numbers)")
input_component_two = gr.Textbox(label="Number of articles")
output_component_one = gr.Textbox(label="Stock Price Prediction")
output_component_two = gr.Textbox(label="Public Sentiment Score ")

output_component = [output_component_one, output_component_two]

interface = gr.Interface(fn= combined_pipelines, inputs=[input_component_one,input_component_two], outputs=[output_component_one, output_component_two])
interface.launch()